In [ ]:
import rospy
import tf
import json
from moveit_msgs.srv import GetPositionIK, GetPositionIKRequest
from geometry_msgs.msg import PoseStamped
from cs_golf.robot import Robot
from cs_golf.trajectories import trapezoidal_speed_trajectory
from copy import deepcopy
from cs_golf.transformations import quat_rotate, multiply_transform, inverse_transform, list_to_pose_stamped
from moveit_msgs.msg import RobotTrajectory
from trajectory_msgs.msg import JointTrajectoryPoint

In [ ]:
rospy.init_node("putter_rotations")

In [ ]:
tfl = tf.TransformListener()

In [ ]:
tfb = tf.TransformBroadcaster()

In [ ]:
# rosrun tf static_transform_publisher -0.385 -0.396 0.02 -0.604 -0.405 -0.601 0.331 iiwa_link_ee putter 50
# roslaunch iiwa_moveit demo.launch driver:=true

In [35]:
DISTANCE_TO_CENTRE = 0.12    # Run-up
TRAJECTORY_RANGE = 0.15      # In-line trajectory distance
HEIGHT_ANGLE_SHOULDER = 0.75 
LAST_TRACE_DISTANCE = 0.1    # Deceleration straight line

In [36]:
world_P_centre = [[0.0 + DISTANCE_TO_CENTRE, -0.55, 0.03], [0, 0, 0, 1]]

In [37]:
gripper_T_putter = [[-0.385, -0.396, 0.02], [-0.604, -0.405, -0.601, 0.331]]
putter_T_gripper = inverse_transform(gripper_T_putter)

In [38]:
ik = rospy.ServiceProxy("/iiwa/compute_ik", GetPositionIK)

In [39]:
robot = Robot()

In [45]:
NUM_ROTATIONS = 100
NUM_POINTS_IN_TRAJ = 20

# Seed is poses["init"]
last_ik_seed_angle = robot.current_state
last_ik_seed_angle.joint_state.position = [-1.5845235972707337, 0.18173555759237864, 0.002154463990176006, -1.8228412945521233, 1.5210518225011864, 1.5430014337672806, 0.04309023940735948]

rospy.sleep(3)
trajectories = []
for rot in range(NUM_ROTATIONS):
    # Compute each putter angle between [0, 0.7[ rad
    angle = 0 + rot*0.6/NUM_ROTATIONS
    rotation = tf.transformations.quaternion_about_axis(angle, (0, 0, 1))
    trajectory = {"angle": angle, "points": []}
    
    world_P_centre_angled = [world_P_centre[0], rotation]
    #tfb.sendTransform(world_P_centre_angled[0], world_P_centre_angled[1], rospy.Time.now(), "centre_rot", "world")
    
    world_P_shoulder = deepcopy(world_P_centre_angled)
    world_P_shoulder[0][2] += HEIGHT_ANGLE_SHOULDER
    
    # rotation movement of pi/4 (= 70/100)
    last_ik_seed = last_ik_seed_angle
    CIRCLE_RANGE = 80
    for rot2 in range(CIRCLE_RANGE):
        # Publish "shoulder" in this loop so that it's always available
        tfb.sendTransform(world_P_shoulder[0], world_P_shoulder[1], rospy.Time.now(), "shoulder", "world")
        angle2 = rot2 / 100. + 0.002*CIRCLE_RANGE
        reverse_angle2 = 0.7 - angle2
        rotation2 = tf.transformations.quaternion_about_axis(reverse_angle2, (0, 1, 0))
        shoulder_T_putter_rot = quat_rotate(rotation2, [0, 0, -HEIGHT_ANGLE_SHOULDER])
        shoulder_T_putter = [shoulder_T_putter_rot, rotation2]
        tfb.sendTransform(shoulder_T_putter[0], shoulder_T_putter[1], rospy.Time.now(), "putter2", "shoulder")
        #rospy.sleep(0.01)

        # Get end effector pose for each angle
        shoulder_P_gripper = multiply_transform(shoulder_T_putter, putter_T_gripper)
        world_P_gripper = multiply_transform(world_P_shoulder, shoulder_P_gripper)

        tfb.sendTransform(world_P_gripper[0], world_P_gripper[1], rospy.Time.now(), "gripper", "world")
        ps_eef = list_to_pose_stamped(world_P_gripper, frame_id="world")
        
        # Get IK for the init pose       
        req = GetPositionIKRequest()
        req.ik_request.group_name = "manipulator"
        req.ik_request.ik_link_name = "iiwa_link_ee"
        req.ik_request.pose_stamped = ps_eef
        req.ik_request.robot_state = last_ik_seed
        result = ik(req)
        if result.error_code.val == -31:
            print("No IK solution for", rot, "angle", angle, "point", point, "distance", distance_to_point)
            break
        else:
            #print(i, angle, j)
            last_ik_seed = result.solution
            #rospy.sleep(0.02)
            trajectory["points"].append(result.solution.joint_state.position)
            if rot2 == 0:
                last_ik_seed_angle = last_ik_seed
    trajectories.append(trajectory)

In [ ]:
   
    continue
    for point in range(NUM_POINTS_IN_TRAJ):
        distance_to_point = float(point)*TRAJECTORY_RANGE/NUM_POINTS_IN_TRAJ
        position = quat_rotate(rotation, [-DISTANCE_TO_CENTRE + distance_to_point, 0, 0])
        centre_T_putter = [position, rotation]

        tfb.sendTransform(world_P_putter[0], world_P_putter[1], rospy.Time.now(), "putter2", "world")

        # Get end effector pose for each angle
        world_P_gripper = multiply_transform(world_P_putter, putter_T_gripper)
        tfb.sendTransform(world_P_gripper[0], world_P_gripper[1], rospy.Time.now(), "gripper", "world")
        ps_eef = list_to_pose_stamped(world_P_gripper, frame_id="world")
        continue

        # Get IK for the init pose
        # /!!!!\ The seed is the current pose
        req = GetPositionIKRequest()
        req.ik_request.group_name = "manipulator"
        req.ik_request.ik_link_name = "iiwa_link_ee"
        req.ik_request.pose_stamped = ps_eef
        req.ik_request.robot_state = last_ik_seed_point
        result = ik(req)
        if result.error_code.val == -31:
            print("No IK solution for", rot, "angle", angle, "point", point, "distance", distance_to_point)
        else:
            #print(i, angle, j)
            last_ik_seed_point = result.solution
            traj = trapezoidal_speed_trajectory(result.solution, last_ik_seed_point, 1, 1)
            #robot.display(traj)
            #rospy.sleep(0.02)
            trajectory["points"].append(result.solution.joint_state.position)
            if point == 0:
                last_ik_seed = last_ik_seed_point
        #ospy.sleep(0.05)
    continue
    trajectories.append(trajectory)

In [43]:
for t in trajectories:
    rt = RobotTrajectory()
    rt.joint_trajectory.joint_names = last_ik_seed.joint_state.name
    for point in t["points"]:
        jtp = JointTrajectoryPoint()
        jtp.positions = point
        rt.joint_trajectory.points.append(jtp)
    robot.display(rt)
    rospy.sleep(0.5)

In [ ]:
with open("../config/motions.json", "w") as f:
    json.dump({"joints": last_ik_seed.joint_state.name, "trajectories": trajectories}, f)

In [46]:
len(trajectories)

100